In [ ]:
# !pip install psycopg2
# !pip install python-dotenv

In [ ]:
import pandas as pd
import psycopg2
import os
from dotenv import load_dotenv, find_dotenv

In [ ]:
load_dotenv(find_dotenv())

In [ ]:
DBNAME = os.getenv('DBNAME')
USER = os.getenv('USER')
PASSWORD = os.getenv('PASSWORD')
HOST = os.getenv('HOST')
PORT = os.getenv('PORT')